In [1]:
import torch
from torch import nn,optim
from torch.utils.data import Dataset,TensorDataset,DataLoader
import tqdm

import numpy as np

from torchvision.datasets import ImageFolder
from torchvision import transforms
from torchvision.utils import save_image
from PIL import Image
import time
import math

### imageの準備

In [2]:
"""
!wget http://www.robots.ox.ac.uk/~vgg/data/flowers/102/102flowers.tgz
!tar xf 102flowers.tgz
!mkdir oxford-102
!mkdir oxford-102/jpg
!mv jpg/*.jpg oxford-102/jpg
"""

'\n!wget http://www.robots.ox.ac.uk/~vgg/data/flowers/102/102flowers.tgz\n!tar xf 102flowers.tgz\n!mkdir oxford-102\n!mkdir oxford-102/jpg\n!mv jpg/*.jpg oxford-102/jpg\n'

# Super Resolution

In [3]:
class DownSizedPairImageFolder(ImageFolder):
    def __init__(self,root,transform=None,large_size = 256,small_size =64,**kwds):
        super().__init__(root,transform = transform, **kwds)
        self.large_resizer = transforms.Resize(large_size)
        self.small_resizer = transforms.Resize(small_size)

    def __getitem__(self,index):
        path,_ = self.imgs[index]
        img = self.loader(path)
        
        cropper = transforms.CenterCrop(500)
        large_img = self.large_resizer(cropper(img))
        small_img = self.small_resizer(cropper(img))

        if self.transform is not None:
            large_img=self.transform(large_img)
            small_img=self.transform(small_img)

        return small_img, large_img

In [4]:
#データローダーの準備
train_data = DownSizedPairImageFolder("./oxford-102/", transform = transforms.ToTensor())
    
test_data = DownSizedPairImageFolder("./oxford-102/", transform = transforms.ToTensor())        

batch_size = 32
train_loader = DataLoader(train_data,batch_size = batch_size,shuffle = True, num_workers = 18)
test_loader = DataLoader(test_data,batch_size = batch_size,shuffle = True, num_workers = 18)

In [5]:
net=nn.Sequential(
    nn.Conv2d(3,256,4,stride=2,padding=1),
    nn.ReLU(),
    nn.BatchNorm2d(256),
    nn.Conv2d(256,512,4,stride=2,padding=1),
    nn.ReLU(),
    nn.BatchNorm2d(512),
    nn.ConvTranspose2d(512,256,4,stride=2,padding=1),
    nn.ReLU(),
    nn.BatchNorm2d(256),
    nn.ConvTranspose2d(256,128,4,stride=2,padding=1),
    nn.ReLU(),
    nn.BatchNorm2d(128),
    nn.ConvTranspose2d(128,64,4,stride=2,padding=1),
    nn.ReLU(),
    nn.BatchNorm2d(64),
    nn.ConvTranspose2d(64,3,4,stride=2,padding=1)
    )

In [6]:
dev = "cpu"#"cuda:0"
net.to(dev)

def psnr(mse,max_v = 1.0):
    return 10*math.log10(max_v**2/mse)

In [7]:
def eval_net(net,data_loader,device = dev):
    net.eval()
    ys = []
    ypreds = []
    for x,y in data_loader:
        x = x.to(device)
        y = y.to(device)
        with torch.no_grad():
            y_pred = net(x)
        ys.append(y)
        ypreds.append(y_pred)
    ys = torch.cat(ys)
    ypreds = torch.cat(ypreds)
    score = nn.functional.mse_loss(ypreds,ys).item()
    return score

In [8]:
def train_net(net,train_loader,test_loader,optimizer_cls = optim.Adam,loss_fn = nn.MSELoss(),n_iter = 20,device = dev):
    train_losses = []
    train_acc = []
    val_acc = []
    optimizer = optimizer_cls(net.parameters())
    for epoch in range(n_iter):
        running_loss = 0.0
        net.train()
        n=0
        score=0
        for i , (xx,yy) in tqdm.tqdm(enumerate(train_loader),total=len(train_loader)):
            xx = xx.to(device)
            yy = yy.to(device)
            y_pred = net(xx)
            loss = loss_fn(y_pred,yy)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            n += len(xx)
            del xx,yy
        train_losses.append(running_loss/len(train_loader))
        val_acc.append(eval_net(net,test_loader,device))
        print(epoch,train_losses[-1],psnr(train_losses[-1]),psnr(val_acc[-1]),flush=True)

In [9]:
train_net(net,train_loader,test_loader,device = dev)

100%|██████████| 256/256 [06:53<00:00,  1.62s/it]


0 0.05749465434928425 12.403725326065638 19.936802888732117


100%|██████████| 256/256 [06:46<00:00,  1.59s/it]


1 0.00805052020768926 20.941760555291022 22.390625748021918


100%|██████████| 256/256 [06:43<00:00,  1.58s/it]


...

100%|██████████| 256/256 [06:57<00:00,  1.63s/it]


18 0.0028079888033971656 25.516046282489707 26.161936952191923


100%|██████████| 256/256 [06:59<00:00,  1.64s/it]


19 0.002663599402694672 25.745310911689046 26.448602330162544


"\nfiles = sorted(glob.glob('./resolution-data/*.jpg'))  \nimages = list(map(lambda file : Image.open(file) , files))\nimages[0].save('Super_resolving_process.gif' , save_all = True , append_images = images[1:] , duration = 400 , loop = 0)\n"

# GAN

In [10]:
img_data = ImageFolder("./oxford-102/",
                     transform=transforms.Compose([transforms.Resize(80),transforms.CenterCrop(64),transforms.ToTensor()]))
batch_size = 64
img_loader = DataLoader(img_data,batch_size = batch_size, shuffle = True)

In [11]:
dev = "cuda:0"

### GnetとDnetの定義

In [12]:
nz = 100
ngf = 32
class GNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.main = nn.Sequential(
            nn.ConvTranspose2d(nz,ngf*8,4,1,0,bias=False),
            nn.BatchNorm2d(ngf*8),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(ngf*8,ngf*4,4,2,1,bias=False),
            nn.BatchNorm2d(ngf*4),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(ngf*4,ngf*2,4,2,1,bias=False),
            nn.BatchNorm2d(ngf*2),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(ngf*2,ngf*1,4,2,1,bias=False),
            nn.BatchNorm2d(ngf*1),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(ngf*1,3,4,2,1,bias=True),
            nn.Tanh()
            )

    def forward(self,x):
        out = self.main(x)
        return out

In [13]:
ndf=32
#(3,64,64)
class DNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.main = nn.Sequential(
            nn.Conv2d(3,ndf,4,2,1,bias=False),
            nn.LeakyReLU(0.2,inplace=True),
            nn.Conv2d(ndf,ndf*2,4,2,1,bias=False),
            nn.BatchNorm2d(ndf*2),
            nn.LeakyReLU(0.2,inplace=True),
            nn.Conv2d(ndf*2,ndf*4,4,2,1,bias=False),
            nn.BatchNorm2d(ndf*4),
            nn.LeakyReLU(0.2,inplace=True),
            nn.Conv2d(ndf*4,ndf*8,4,2,1,bias=False),
            nn.BatchNorm2d(ndf*8),
            nn.LeakyReLU(0.2,inplace=True),
            nn.Conv2d(ndf*8,1,4,1,0,bias=True)
            )

    def forward(self,x):
        out = self.main(x)
        return out.squeeze()

In [14]:
d = DNet().to(dev)
g = GNet().to(dev)

opt_d = optim.Adam(d.parameters(),lr=0.0002,betas=(0.5,0.999))
opt_g = optim.Adam(g.parameters(),lr=0.0002,betas=(0.5,0.999))

ones = torch.ones(batch_size).to(dev)
zeros = torch.zeros(batch_size).to(dev)

loss_f = nn.BCEWithLogitsLoss()

fixed_z = torch.randn(batch_size,nz,1,1).to(dev)

### 訓練関数の定義

In [15]:
from statistics import mean

def train_dcgan(g,d,opt_g,opt_d,loader):
    log_loss_g = []
    log_loss_d = []
    
    for real_img , _ in tqdm.tqdm(loader):
        batch_len = len(real_img)
        #print(real_img.size())
        real_img = real_img.to(dev)
        z = torch.randn(batch_len,nz,1,1).to(dev)
        #------------------------------------------------------------------------------------------------------------------
        fake_img = g(z)                                                         #フェイクイメージの作成
        fake_img_tensor = fake_img.detach()                  #フェイクイメージを計算グラフから分離
        out = d.forward(fake_img)                                      #フェイクイメージの判定
        loss_g = loss_f(out, ones[:batch_len])                 #フェイク画像の損失計算 as 本物
        log_loss_g.append(loss_g.item())                        #
        d.zero_grad() , g.zero_grad()                               #勾配初期化
        loss_g.backward()                                                  #生成機の逆伝搬
        opt_g.step()                                                           #生成機の更新
        #------------------------------------------------------------------------------------------------------------------
        real_out = d.forward(real_img)                                       #本物画像の判定
        loss_d_real = loss_f(real_out,ones[:batch_len])         #本物画像の損失計算

        fake_img = fake_img_tensor                                          #
        fake_out = d.forward(fake_img_tensor)                      #
        loss_d_fake = loss_f(fake_out, zeros[:batch_len])    #フェイク画像の損失計算 as フェイク

        loss_d = loss_d_real + loss_d_fake
        log_loss_d.append(loss_d.item())
        d.zero_grad(),g.zero_grad()                                         #勾配初期化
        loss_d.backward()                                                          #判別機の逆伝搬
        opt_d.step()                                                                   #判別機の更新
        #------------------------------------------------------------------------------------------------------------------
    return mean(log_loss_g),mean(log_loss_d)

### 訓練開始

In [16]:
import os
if not os.path.exists("data"):
    os.mkdir("data")
if not os.path.exists("data2"):
    os.mkdir("data2")

In [17]:
import glob

total_time=0.0
average_time=0.0
iter_epoch=100


for epoch in range(iter_epoch):
    t_start=time.time()
    print("current epoch : ",epoch)
    train_dcgan(g,d,opt_g,opt_d,img_loader)
    t_finish=time.time()
    total_time+=t_finish-t_start
    average_time=total_time/(epoch+1)
    print("runtime = ",t_finish-t_start,"sec")
    print("expected remaining time = ",int(average_time*(iter_epoch-(epoch+1))),"sec",":",int(average_time*(iter_epoch-(epoch+1))/60),"minute")
    #gif_data.append(g(gif_z))
    if epoch%1==0:
        torch.save(g.state_dict(),"./data/g_{:03d}.prm".format(epoch),pickle_protocol=4)
        torch.save(d.state_dict(),"./data/d_{:03d}.prm".format(epoch),pickle_protocol=4)
        generated_img_tmp=g(fixed_z)
        generated_img=net(generated_img_tmp.to("cpu"))
        #g = g.to(dev)
        save_image(generated_img,"./data/{:03d}.jpg".format(epoch))
        save_image(generated_img_tmp,"./data2/{:03d}.jpg".format(epoch))
        


files = sorted(glob.glob('./data/*.jpg'))  
images = list(map(lambda file : Image.open(file) , files))
images[0].save('generating_process.gif' , save_all = True , append_images = images[1:] , duration = 200 , loop = 0)

  0%|          | 0/128 [00:00<?, ?it/s]

current epoch :  0


100%|██████████| 128/128 [00:38<00:00,  3.33it/s]


runtime =  38.396382093429565 sec
expected remaining time =  3801 sec : 63 minute


  0%|          | 0/128 [00:00<?, ?it/s]

...

runtime =  37.65106272697449 sec
expected remaining time =  38 sec : 0 minute


  0%|          | 0/128 [00:00<?, ?it/s]

current epoch :  99


100%|██████████| 128/128 [00:38<00:00,  3.32it/s]


runtime =  38.59035611152649 sec
expected remaining time =  0 sec : 0 minute
